In [1]:
################### 买入后 上涨 大于30 的训练  
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [2]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

######### m 为样本个数 ###############
def J_sigmond(h,y,m):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / m

In [3]:
###### 读取数据 ########
stock_data_org = pd.read_csv('stock_data_normal_u.csv',header=None)
stock_lable_org = pd.read_csv('stock_data_lable_normal.txt',header=None)

In [4]:
print(stock_data_org.shape[0])
print(stock_data_org.shape[0] / 6)
print(stock_lable_org.shape[0])

130116
21686.0
21686


In [5]:
################# 抽取出成功的标签，使成功与失败的比例是1:1.2 #####################
######### 所有的 股票数据，转换成（m,66 的形势）
all_x_data = stock_data_org.values
all_x_data = all_x_data.reshape(-1,66)
all_x_data = np.insert(all_x_data,0,1,axis=1)

################ 为 lable 加上 index
stock_lable_org['index'] = pd.DataFrame(np.arange(stock_lable_org.shape[0] -1))

############### 取出 所有 多空中 为 1 的数据 #####################
stock_lable_1 = stock_lable_org[stock_lable_org[2]==1]
############### 取出 所有 多空中 为 0 的数据 #####################
stock_lable_0 = stock_lable_org[stock_lable_org[2]==0] ##########
############### 对 多空为0 的数据进行抽样，数量为 多空为1 的1.2倍
stock_lable_0_sample = stock_lable_0.sample(n=int(stock_lable_1.shape[0]*1.2),replace=False)
############### 组合 0 的抽样数据与 1 的lable【所有数据】，形成 神经网络 的【训练集】与【测试集】
lable_nn_data = pd.concat([stock_lable_1,stock_lable_0_sample])
lable_nn_data = lable_nn_data.sort_values(by = 'index',ascending=True)
cut_idx = int(lable_nn_data.shape[0] * 0.7)
lable_nn_train = lable_nn_data.iloc[:cut_idx]
lable_nn_test = lable_nn_data.iloc[cut_idx:]
#print(lable_nn_test)
############### 根据 index 取出 所有对应的 股票数据 #####################
train_data_index = lable_nn_train['index'].values
#print(train_data_index)
test_data_index = lable_nn_test['index'].values
#print(test_data_index)
train_data_index = train_data_index.astype(int)
test_data_index = test_data_index.astype(int)
#print(test_data_index)
#print(test_data_index.tolist())
train_data = all_x_data[train_data_index.tolist()]
test_data = all_x_data[test_data_index.tolist()]
#train_data = np.insert(train_data,0,1,axis=1)
#test_data = np.insert(test_data,0,1,axis=1)
y_train = lable_nn_train[2].values.reshape(-1,1)
y_test = lable_nn_test[2].values.reshape(-1,1)

In [6]:
print(train_data.shape[0])
print(y_train.shape[0])
print(test_data.shape[0])
print(y_test.shape[0])

7828
7828
3356
3356


In [7]:
############## 验证数据是否取正确 ################
for i in range(10):
    random_index = random.randint(0,train_data_index.shape[0]-1)
    check_index = train_data_index[random_index]
    print('输入x',train_data[random_index])
    print('输入y',y_train[random_index])
    print('原有x',all_x_data[check_index])
    print('原有y',stock_lable_org.iloc[check_index])

输入x [ 1.      -1.13251 -1.25424 -1.2065  -1.19029 -0.14657 -1.19703 -1.10393
 -0.8802  -0.78479 -0.79665 -0.98503 -0.76008 -1.16494 -1.16879 -0.81727
 -0.54997 -1.24528 -0.91063 -0.92468 -0.96211 -0.88566 -0.91599 -0.76008
  0.0203  -0.11311 -0.67739  1.6221   0.11872 -0.44028 -0.5712  -0.61404
 -0.61863 -0.52934  0.59612  0.58044  0.57309  0.73697  0.28119  0.90798
  0.17826  0.01873 -0.03612 -0.10236  0.09206  0.81396  1.10811  0.91995
  0.88462  0.18327  1.00437  0.87842  0.81466  0.87674  0.81445  0.87916
  1.2426   0.71033  0.99536  1.06336 -1.39003  0.41123  1.39816  1.5427
  1.52033  1.58885  1.45914]
输入y [1]
原有x [ 1.      -1.13251 -1.25424 -1.2065  -1.19029 -0.14657 -1.19703 -1.10393
 -0.8802  -0.78479 -0.79665 -0.98503 -0.76008 -1.16494 -1.16879 -0.81727
 -0.54997 -1.24528 -0.91063 -0.92468 -0.96211 -0.88566 -0.91599 -0.76008
  0.0203  -0.11311 -0.67739  1.6221   0.11872 -0.44028 -0.5712  -0.61404
 -0.61863 -0.52934  0.59612  0.58044  0.57309  0.73697  0.28119  0.90798
  0.178

In [10]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1.txt",delimiter=',')
w2 = np.loadtxt("init_w2.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 #########
n = 0.5
######### 样本个数 ###########
m = train_data.shape[0] 
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[-0.14872006 -0.8890755  -0.76336209 -0.27224645 -0.41948583]
 [-0.49379923  0.21599772  0.30278697 -0.69725246  0.77879905]
 [ 0.55388879 -0.32684416 -0.62443618  0.76399624  0.22638863]
 [ 0.30948447  0.03173112  0.96560106  0.69256066 -0.13071171]
 [ 0.7822694  -0.65191623 -0.49056081  0.58718041  0.93831124]
 [ 0.84425725  0.88626155 -0.19749617  0.48648008 -0.18920059]
 [ 0.6761767  -0.69972915  0.2486178   0.20556349  0.08308843]
 [ 0.59952789  0.81383611 -0.67258272  0.84252215  0.04881861]
 [ 0.51353092  0.85224435 -0.13340034  0.72049805  0.24497916]
 [-0.02725946  0.79379224  0.88296628 -0.50035128  0.52424028]]
[[-0.7940387 ]
 [-0.74160387]
 [ 0.41070837]
 [-0.17728893]
 [ 0.37561007]
 [ 0.53012985]
 [ 0.02742154]
 [ 0.5203346 ]
 [-0.86921099]
 [-0.57189687]]


In [12]:
for i in range(2000):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(train_data,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y_train
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / m
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(train_data.T,d_cost_hid_input) / m
    print('----------cost is -------------',J_sigmond(hpfs_output,y_train,train_data.shape[0]))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')

print('----------------done--------------------------')

----------cost is ------------- 0.37140185749348703
----------cost is ------------- 0.3640545621293994
----------cost is ------------- 0.3712872905274184
----------cost is ------------- 0.363933489772302
----------cost is ------------- 0.3711728704811687
----------cost is ------------- 0.3638125625394329
----------cost is ------------- 0.3710585963946586
----------cost is ------------- 0.36369177958037724
----------cost is ------------- 0.370944467250472
----------cost is ------------- 0.36357113999428947
----------cost is ------------- 0.3708304819751383
----------cost is ------------- 0.36345064283096884
----------cost is ------------- 0.3707166394404699
----------cost is ------------- 0.3633302870919836
----------cost is ------------- 0.37060293846494907
----------cost is ------------- 0.363210071731844
----------cost is ------------- 0.3704893778151628
----------cost is ------------- 0.3630899956592181
----------cost is ------------- 0.37037595620728203
----------cost is ----------

----------cost is ------------- 0.36256831903861236
----------cost is ------------- 0.35470990900106864
----------cost is ------------- 0.36246017637875727
----------cost is ------------- 0.35459571798149064
----------cost is ------------- 0.362352043938554
----------cost is ------------- 0.3544815507757138
----------cost is ------------- 0.36224392042369724
----------cost is ------------- 0.35436740624266816
----------cost is ------------- 0.36213580456268485
----------cost is ------------- 0.3542532832621002
----------cost is ------------- 0.36202769510668703
----------cost is ------------- 0.3541391807344624
----------cost is ------------- 0.361919590829399
----------cost is ------------- 0.3540250975807888
----------cost is ------------- 0.3618114905268794
----------cost is ------------- 0.3539110327425557
----------cost is ------------- 0.361703393017371
----------cost is ------------- 0.35379698518152775
----------cost is ------------- 0.36159529714111044
----------cost is ------

----------cost is ------------- 0.35376306752984366
----------cost is ------------- 0.34546928602550503
----------cost is ------------- 0.3536530797507726
----------cost is ------------- 0.34535472571967596
----------cost is ------------- 0.35354304884392934
----------cost is ------------- 0.34524014426148997
----------cost is ------------- 0.353432974513775
----------cost is ------------- 0.34512554143199814
----------cost is ------------- 0.3533228564687311
----------cost is ------------- 0.3450109170157051
----------cost is ------------- 0.35321269442111547
----------cost is ------------- 0.34489627080048824
----------cost is ------------- 0.3531024880870825
----------cost is ------------- 0.3447816025775213
----------cost is ------------- 0.3529922371865673
----------cost is ------------- 0.3446669121412014
----------cost is ------------- 0.3528819414432336
----------cost is ------------- 0.34455219928907743
----------cost is ------------- 0.3527716005844258
----------cost is -----

----------cost is ------------- 0.34480894621085223
----------cost is ------------- 0.33622378841882444
----------cost is ------------- 0.3446948513724942
----------cost is ------------- 0.3361070463743274
----------cost is ------------- 0.3445806999691496
----------cost is ------------- 0.3359902726167083
----------cost is ------------- 0.3444664919379437
----------cost is ------------- 0.3358734670637356
----------cost is ------------- 0.34435222721768866
----------cost is ------------- 0.33575662963354913
----------cost is ------------- 0.3442379057488434
----------cost is ------------- 0.3356397602446371
----------cost is ------------- 0.34412352747347535
----------cost is ------------- 0.33552285881581445
----------cost is ------------- 0.3440090923352214
----------cost is ------------- 0.3354059252662016
----------cost is ------------- 0.3438946002792501
----------cost is ------------- 0.33528895951520393
----------cost is ------------- 0.343780051252224
----------cost is -------

----------cost is ------------- 0.33549932025862633
----------cost is ------------- 0.32677769323363
----------cost is ------------- 0.3353805899933094
----------cost is ------------- 0.3266581639387388
----------cost is ------------- 0.3352618010228755
----------cost is ------------- 0.32653859658696366
----------cost is ------------- 0.33514295333515903
----------cost is ------------- 0.32641899109725886
----------cost is ------------- 0.3350240469179686
----------cost is ------------- 0.32629934738850497
----------cost is ------------- 0.33490508175907757
----------cost is ------------- 0.32617966537950865
----------cost is ------------- 0.33478605784621546
----------cost is ------------- 0.32605994498900326
----------cost is ------------- 0.3346669751670598
----------cost is ------------- 0.3259401861356498
----------cost is ------------- 0.33454783370922747
----------cost is ------------- 0.3258203887380381
----------cost is ------------- 0.33442863346026713
----------cost is ----

...

----------cost is ------------- 0.294436545662474
----------cost is ------------- 0.2862190244863322
----------cost is ------------- 0.2943004854627931
----------cost is ------------- 0.2860862780560947
----------cost is ------------- 0.29416438459711297
----------cost is ------------- 0.28595350014647963
----------cost is ------------- 0.29402824311490605
----------cost is ------------- 0.2858206908191779
----------cost is ------------- 0.2938920610642157
----------cost is ------------- 0.28568785013473413
----------cost is ------------- 0.29375583849164244
----------cost is ------------- 0.2855549781525286
----------cost is ------------- 0.2936195754423288
----------cost is ------------- 0.28542207493075705
----------cost is ------------- 0.2934832719599449
----------cost is ------------- 0.2852891405264135
----------cost is ------------- 0.2933469280866749
----------cost is ------------- 0.28515617499527096
----------cost is ------------- 0.29321054386320233
----------cost is ------

----------cost is ------------- 0.2834247633463997
----------cost is ------------- 0.27550271180413527
----------cost is ------------- 0.28328547333051024
----------cost is ------------- 0.2753675247717641
----------cost is ------------- 0.2831461414701194
----------cost is ------------- 0.2752323065485763
----------cost is ------------- 0.2830067676899265
----------cost is ------------- 0.2750970570778137
----------cost is ------------- 0.28286735191371554
----------cost is ------------- 0.27496177630163215
----------cost is ------------- 0.2827278940643932
----------cost is ------------- 0.27482646416113177
----------cost is ------------- 0.2825883940640292
----------cost is ------------- 0.2746911205963891
----------cost is ------------- 0.28244885183389395
----------cost is ------------- 0.27455574554648904
----------cost is ------------- 0.28230926729450073
----------cost is ------------- 0.27442033894955803
----------cost is ------------- 0.2821696403656459
----------cost is ----

----------cost is ------------- 0.2721427057708877
----------cost is ------------- 0.2645836328892982
----------cost is ------------- 0.2719998206132334
----------cost is ------------- 0.26444573596435733
----------cost is ------------- 0.27185688799832075
----------cost is ------------- 0.26430780259544334
----------cost is ------------- 0.27171390789379113
----------cost is ------------- 0.26416983273434774
----------cost is ------------- 0.2715708802684759
----------cost is ------------- 0.2640318263334821
----------cost is ------------- 0.2714278050923781
----------cost is ------------- 0.2638937833458623
----------cost is ------------- 0.2712846823366555
----------cost is ------------- 0.2637557037250923
----------cost is ------------- 0.27114151197359887
----------cost is ------------- 0.2636175874253464
----------cost is ------------- 0.27099829397661324
----------cost is ------------- 0.2634794344013507
----------cost is ------------- 0.27085502832019476
----------cost is -----

----------cost is ------------- 0.2605605054366982
----------cost is ------------- 0.25343318137964166
----------cost is ------------- 0.2604137753625933
----------cost is ------------- 0.2532922301941007
----------cost is ------------- 0.2602669966685954
----------cost is ------------- 0.25315123870006334
----------cost is ------------- 0.26012016932799437
----------cost is ------------- 0.25301020682477665
----------cost is ------------- 0.25997329331325403
----------cost is ------------- 0.2528691344945046
----------cost is ------------- 0.25982636859600583
----------cost is ------------- 0.25272802163452723
----------cost is ------------- 0.25967939514704347
----------cost is ------------- 0.2525868681691389
----------cost is ------------- 0.25953237293631864
----------cost is ------------- 0.25244567402164714
----------cost is ------------- 0.25938530193293646
----------cost is ------------- 0.2523044391143726
----------cost is ------------- 0.25923818210515215
----------cost is -

----------cost is ------------- 0.24866494299176348
----------cost is ------------- 0.24202223779731802
----------cost is ------------- 0.24851417103455675
----------cost is ------------- 0.24187775590818528
----------cost is ------------- 0.24836334577630625
----------cost is ------------- 0.24173322507374007
----------cost is ------------- 0.24821246712969625
----------cost is ------------- 0.24158864515571055
----------cost is ------------- 0.24806153500678818
----------cost is ------------- 0.24144401601520377
----------cost is ------------- 0.24791054931902137
----------cost is ------------- 0.24129933751270793
----------cost is ------------- 0.2477595099772149
----------cost is ------------- 0.24115460950809525
----------cost is ------------- 0.2476084168915679
----------cost is ------------- 0.2410098318606243
----------cost is ------------- 0.24745726997166054
----------cost is ------------- 0.24086500442894238
----------cost is ------------- 0.24730606912645559
----------cost 

In [ ]:
################# 先用 验证集进行验证 ##################
test_error_count = 0
#print(test_data.shape[0])
for i in range(test_data.shape[0]-1):
    x_test_data = test_data[i]
    hid_input = np.matmul(x_test_data,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=0)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    #print(y_test.shape)
    if (hpfs_output>0.5 and y_test[i] ==0 ) or (hpfs_output <0.5 and y_test[i] ==1):
        test_error_count = test_error_count +1
        print(hpfs_output,y_test[i])
print(test_error_count/test_data.shape[0])

In [ ]:
################# 用所有数据验证 ##################
all_error_count = 0
#all_x_data = np.insert(all_x_data,0,1,axis=1)
#print(test_data.shape[0])
for i in range(all_x_data.shape[0]-1):
    x_test_data = all_x_data[i]
    hid_input = np.matmul(x_test_data,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=0)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    #print(y_test.shape)
    if (hpfs_output>0.5 and stock_lable_org.iloc[i,2] ==0 ) or (hpfs_output <0.5 and stock_lable_org.iloc[i,2] ==1):
        all_error_count = all_error_count +1
        print(hpfs_output,stock_lable_org.iloc[i,2])
print(all_error_count/all_x_data.shape[0])